In [41]:
from mxnet import autograd
from mxnet import gluon
from mxnet import image
from mxnet import init
from mxnet import nd
from mxnet.gluon.data import vision
import numpy as np

def transform_train(data, label):
    im = image.imresize(data.astype('float32') / 255, 96, 96)
    auglist = image.CreateAugmenter(data_shape=(3, 96, 96), resize=0,
                        rand_crop=False, rand_resize=False, rand_mirror=True,
                        mean=None, std=None,
                        brightness=0, contrast=0,
                        saturation=0, hue=0,
                        pca_noise=0, rand_gray=0, inter_method=2)
    for aug in auglist:
        im = aug(im)
    # 将数据格式从"高*宽*通道"改为"通道*高*宽"。
    im = nd.transpose(im, (2,0,1))
    return (im, nd.array([label]).asscalar().astype('float32'))

def transform_test(data, label):
    im = image.imresize(data.astype('float32') / 255, 96, 96)
    im = nd.transpose(im, (2,0,1))
    return (im, nd.array([label]).asscalar().astype('float32'))

In [42]:
batch_size = 32

train_ds = vision.ImageFolderDataset('Images', flag=1,
                                     transform=transform_train)
valid_ds = vision.ImageFolderDataset('valid', flag=1,
                                     transform=transform_test)
test_ds = vision.ImageFolderDataset('test', flag=1,
                                     transform=transform_test)
loader = gluon.data.DataLoader
train_data = loader(train_ds, batch_size, shuffle = True, last_batch = 'keep')
valid_data = loader(valid_ds, batch_size, shuffle = True, last_batch = 'keep')
test_data = loader(test_ds, batch_size, shuffle = False, last_batch = 'keep')

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [43]:
from mxnet import init
from mxnet.gluon.model_zoo import vision as models

def get_net(ctx):
    pretrained_net = models.resnet152_v1(pretrained = True, ctx = ctx)
    finetune_net = models.resnet152_v1(classes=120)
    finetune_net.features = pretrained_net.features
    finetune_net.output.initialize(ctx = ctx, init = init.Xavier())
    return finetune_net

In [44]:
import datetime
import sys
sys.path.append('..')
import utils

def get_loss(data, net, ctx):
    loss = 0.0
    for feas, label in data:
        label = label.as_in_context(ctx)
        output = net(feas.as_in_context(ctx))
        cross_entropy = softmax_cross_entropy(output, label)
        loss += nd.mean(cross_entropy).asscalar()
    return loss / len(data)

def train(net, train_data, valid_data, num_epochs, lr, wd, ctx, lr_period,
          lr_decay):
    trainer = gluon.Trainer(
        net.collect_params(), 'sgd', {'learning_rate': lr, 'momentum': 0.9,
                                      'wd': wd})
    prev_time = datetime.datetime.now()
    for epoch in range(num_epochs):
        train_loss = 0.0
        if epoch > 0 and epoch % lr_period == 0:
            trainer.set_learning_rate(trainer.learning_rate * lr_decay)
        for data, label in train_data:
            label = label.as_in_context(ctx)
            with autograd.record():
                output = net(data.as_in_context(ctx))
                loss = softmax_cross_entropy(output, label)
            loss.backward()
            trainer.step(batch_size)
            train_loss += nd.mean(loss).asscalar()
        cur_time = datetime.datetime.now()
        h, remainder = divmod((cur_time - prev_time).seconds, 3600)
        m, s = divmod(remainder, 60)
        time_str = "Time %02d:%02d:%02d" % (h, m, s)
        if valid_data is not None:
            valid_loss = get_loss(valid_data, net, ctx)
            epoch_str = ("Epoch %d. Train loss: %f, Valid loss %f, "
                         % (epoch, train_loss / len(train_data), valid_loss))
        else:
            epoch_str = ("Epoch %d. Train loss: %f, "
                         % (epoch, train_loss / len(train_data)))
        prev_time = cur_time
        print(epoch_str + time_str + ', lr ' + str(trainer.learning_rate))

In [45]:

ctx = utils.try_gpu()
num_epochs = 150
learning_rate = 0.01
weight_decay = 5e-4
lr_period = 80
lr_decay = 0.1

net = get_net(ctx)
net.hybridize()
train(net, train_data, valid_data, num_epochs, learning_rate,
      weight_decay, ctx, lr_period, lr_decay)


Epoch 0. Train loss: 3.362387, Valid loss 2.522207, Time 00:02:55, lr 0.01
Epoch 1. Train loss: 2.268164, Valid loss 2.019045, Time 00:02:58, lr 0.01
Epoch 2. Train loss: 1.876123, Valid loss 1.778745, Time 00:02:59, lr 0.01
Epoch 3. Train loss: 1.584600, Valid loss 1.549433, Time 00:02:59, lr 0.01
Epoch 4. Train loss: 1.383385, Valid loss 1.405398, Time 00:02:59, lr 0.01
Epoch 5. Train loss: 1.229195, Valid loss 1.202177, Time 00:02:59, lr 0.01
Epoch 6. Train loss: 1.100012, Valid loss 1.085701, Time 00:02:59, lr 0.01
Epoch 7. Train loss: 0.984265, Valid loss 1.202698, Time 00:02:59, lr 0.01
Epoch 8. Train loss: 0.878815, Valid loss 1.007236, Time 00:02:59, lr 0.01
Epoch 9. Train loss: 0.798801, Valid loss 0.932399, Time 00:02:59, lr 0.01
Epoch 10. Train loss: 0.738873, Valid loss 0.851047, Time 00:02:59, lr 0.01
Epoch 11. Train loss: 0.677639, Valid loss 0.761516, Time 00:02:59, lr 0.01
Epoch 12. Train loss: 0.632882, Valid loss 0.777367, Time 00:02:59, lr 0.01
Epoch 13. Train loss: 

Epoch 108. Train loss: 0.008268, Valid loss 0.004867, Time 00:03:00, lr 0.001
Epoch 109. Train loss: 0.010253, Valid loss 0.004964, Time 00:03:03, lr 0.001
Epoch 110. Train loss: 0.008072, Valid loss 0.005694, Time 00:03:05, lr 0.001
Epoch 111. Train loss: 0.008296, Valid loss 0.005489, Time 00:03:02, lr 0.001
Epoch 112. Train loss: 0.008473, Valid loss 0.005402, Time 00:03:03, lr 0.001
Epoch 113. Train loss: 0.008332, Valid loss 0.005172, Time 00:03:01, lr 0.001
Epoch 114. Train loss: 0.009352, Valid loss 0.005218, Time 00:02:59, lr 0.001
Epoch 115. Train loss: 0.008194, Valid loss 0.004572, Time 00:03:03, lr 0.001
Epoch 116. Train loss: 0.008071, Valid loss 0.004177, Time 00:03:01, lr 0.001
Epoch 117. Train loss: 0.008479, Valid loss 0.004276, Time 00:02:58, lr 0.001
Epoch 118. Train loss: 0.008366, Valid loss 0.004151, Time 00:02:58, lr 0.001
Epoch 119. Train loss: 0.008245, Valid loss 0.004390, Time 00:02:58, lr 0.001
Epoch 120. Train loss: 0.008534, Valid loss 0.004425, Time 00:02

In [46]:
import numpy as np
import os

outputs = []
for data, label in test_data:
    output = nd.softmax(net(data.as_in_context(ctx)))
    outputs.extend(output.asnumpy())

In [47]:
ids = sorted(os.listdir('test/unknown'))
with open('submission.csv', 'w') as f:
    f.write('id,' + ','.join(train_ds.synsets) + '\n')
    for i, output in zip(ids, outputs):
        f.write(i.split('.')[0] + ',' + ','.join(
            [str(num) for num in output]) + '\n')